# Memory Storage

[![Open in Colab](https://img.shields.io/badge/Open%20in%20Colab-F9AB00?style=for-the-badge&logo=googlecolab&color=525252)](https://colab.research.google.com/github/H-IAAC/CST-Python/blob/main/examples/Memory%20Storage.ipynb) [![Open in Github](https://img.shields.io/badge/Open%20in%20Github-100000?style=for-the-badge&logo=github&logoColor=white)](https://github.com/H-IAAC/CST-Python/blob/main/examples/Memory%20Storage.ipynb)

In this example, we gonna use the Memory Storage to synchonize memories across two CST instances.

First, we need to ensure that `cst_python` and `redis` is installed:

In [1]:
try:
    import cst_python as cst
    import redis
except:
    !python3 -m pip install cst_python[memory_storage]    

We also need to have a running Redis server. If you're running this notebook in Google Colab, the following cell will install and start Redis:

In [2]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    # Install Redis
    !curl -fsSL https://packages.redis.io/redis-stack/redis-stack-server-6.2.6-v7.focal.x86_64.tar.gz -o redis-stack-server.tar.gz 
    !tar -xvf redis-stack-server.tar.gz

    # Start Redis server
    !./redis-stack-server-6.2.6-v7/bin/redis-stack-server --daemonize yes

We can then import the modules and get started.

In [3]:
import time

import redis

import cst_python as cst
from cst_python.memory_storage import MemoryStorageCodelet

We gonna clean the Redis database to ensure the example works correctly:

In [4]:
redis.Redis().flushall()

True

Our example will involve two CST instances, two nodes, with a memory called "MyMemory" being synchronized between them. Let's start by creating the instance's mind and its memory:

In [5]:
firstnode_mind = cst.Mind()
firstnode_memory = firstnode_mind.create_memory_object("MyMemory", "")

To use memory storage, each node needs to have a MemoryStorageCodelet running in its mind. Let's create the codelet, add the mind and start the mind:

In [6]:
firstnode_mscodelet = MemoryStorageCodelet(firstnode_mind)
firstnode_mscodelet.time_step = 50
firstnode_mind.insert_codelet(firstnode_mscodelet)

firstnode_mind.start()

Let's initialize the memory with an info:

In [7]:
firstnode_memory.set_info("First node info")
firstnode_memory.get_info()

'First node info'

And create the mind and memory of the second node. Notice that its memory is not initialized:

In [8]:
secondnode_mind = cst.Mind()
secondnode_memory = secondnode_mind.create_memory_object("MyMemory", "")

secondnode_memory.get_info()

''

We then create the MemoryStorage of the second instance and start it:

In [9]:
secondnode_mscodelet = MemoryStorageCodelet(secondnode_mind)
secondnode_mscodelet.time_step = 50
secondnode_mind.insert_codelet(secondnode_mscodelet)

secondnode_mind.start()

We wait a while to ensure that the codelet will be executed, and we check the data in the second instance:

In [10]:
time.sleep(0.150)

secondnode_memory.get_info()

'First node info'

We can see that the data has been synchronized!

The process works both ways:

In [11]:
secondnode_memory.set_info("Second node info")
time.sleep(0.150)
firstnode_memory.get_info()

'Second node info'

And it can contain data of a few different types:

In [12]:
firstnode_memory.set_info([1, 2, 3])
time.sleep(0.150)
secondnode_memory.get_info()

[1, 2, 3]

In this example, we used two CST-Python instances in the same machine. But, it could be a CST-Python with a CST-Java instance, or instances in different machines, or even more than two instances.

The [Memory Storage Documentation](https://h-iaac.github.io/CST-Python/_build/html/src/Memory%20Storage.html) contains more information about how the Memory Storage works.